In [3]:
import pandas as pd

In [18]:
methyldata = pd.read_csv("methyl.csv",index_col=0)
exprdata = pd.read_csv("expr.csv",index_col=0)
assigndata = pd.read_csv("assign.csv",usecols=[1,2],index_col=0)
assigndata

,cluster.id
subjects,
subject1,1
subject2,2
subject3,1
subject4,1
subject5,1
...,...
subject496,1
subject497,1
subject498,1


In [19]:
exprdata

,ACACA,ACVRL1,AKT1,AKT1S1,ANXA1,AR,ARAF,ASNS,ATM,BAD,...,VHL,WWTR1,XBP1,XRCC1,XRCC5,YAP1,YBX1,YWHAB,YWHAE,YWHAZ
subject1,-0.778773,6.661350,0.601915,-1.381072,0.276366,6.826198,5.217787,-0.531270,-0.685574,4.965979,...,4.997389,5.227471,-1.173942,4.990185,4.739730,0.752404,4.925204,-0.332289,4.740385,1.588527
subject2,-1.202237,6.599935,-1.059473,-1.343091,0.567053,2.980298,-1.060058,-0.543807,4.112408,-0.377627,...,5.324622,-0.575675,-0.521891,1.270019,5.396299,1.926090,0.432328,0.453741,0.124431,1.516044
subject3,-0.351933,7.037106,-0.169862,-0.600338,0.467504,4.409149,5.523396,-0.820251,-0.896702,6.273091,...,6.548016,6.983861,1.174447,5.292834,5.405190,1.720668,5.243290,0.383033,7.387739,1.114908
subject4,-0.560752,6.302922,-1.095674,-0.335570,-1.799302,4.858635,4.789491,-0.335323,-1.509982,5.800878,...,4.780794,6.124618,-1.631147,5.213470,5.813819,0.552651,4.457551,-0.836184,5.890996,-0.108535
subject5,-0.260576,5.974995,-1.597704,-1.239989,-0.848312,6.202194,3.625550,-0.896659,-0.772299,5.427887,...,5.509940,5.235778,-1.466720,5.391635,5.309465,1.113457,5.062244,-1.151481,5.118148,-0.504209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
subject496,-0.933188,6.758515,0.141599,1.175414,-0.199922,5.404964,4.587526,-2.250117,-0.043880,4.791108,...,4.960378,4.375743,-0.822462,6.340625,5.713636,1.483030,5.739724,0.294931,5.237734,1.493332
subject497,-0.351743,6.307647,-0.395698,-0.410632,-0.335842,5.095289,5.294062,0.064668,-0.599051,4.903005,...,6.760614,6.283541,0.793421,5.089189,4.882482,0.584146,5.546557,-0.167474,5.152645,1.066775
subject498,-0.555563,6.201945,0.369510,-0.527820,1.141942,4.788988,4.594360,-1.039993,-0.339906,6.553227,...,4.372792,5.008954,-0.950863,4.932677,5.589072,1.423199,4.820329,0.054766,6.264235,0.935781
subject499,-0.496976,6.736355,0.309428,1.002468,-1.421916,0.913188,0.544914,-2.264783,4.140739,1.007924,...,4.496152,0.002271,-0.513729,1.362927,5.841151,0.990882,-0.240316,0.078770,0.380601,1.431541


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Leer datos
expression_data = pd.read_csv('expr.csv')
methylation_data = pd.read_csv('methyl.csv')

expression_data = expression_data.iloc[:, 1:]
methylation_data = methylation_data.iloc[:, 1:]
# Convertir todas las columnas a tipo float
expression_data = expression_data.apply(pd.to_numeric, errors='coerce')
methylation_data = methylation_data.apply(pd.to_numeric, errors='coerce')
# Lidiar con valores NaN (si los hay). Aquí simplemente los lleno con 0, pero podrías manejarlo de otra manera.
expression_data.fillna(0, inplace=True)
methylation_data.fillna(0, inplace=True)

expression_data = expression_data.values
methylation_data = methylation_data.values
# Asegurar que tienes el mismo número de muestras en ambos conjuntos de datos


assert expression_data.shape[0] == methylation_data.shape[0], "Los datos de expresión y metilación deben tener el mismo número de muestras."


# Concatenar los datos
combined_data = torch.FloatTensor(np.hstack((expression_data, methylation_data)))

expression_data = torch.FloatTensor(expression_data)
methylation_data = torch.FloatTensor(methylation_data)
# Definir el generador y el discriminador
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim, embed_size, heads):
        super(Generator, self).__init__()
        
        # Inicio de la red
        self.start = nn.Sequential(
            nn.Linear(input_dim, embed_size),
            nn.ReLU()
        )
        
        # Capa de atención del Transformer
        self.attention = SelfAttention(embed_size, heads)
        
        # Salida
        self.end = nn.Sequential(
            nn.Linear(embed_size, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.start(x)
        # Agregar una dimensión adicional para tratar la entrada como una "secuencia"
        x = x.unsqueeze(1)
        x = self.attention(x, x, x)  
        # Eliminar la dimensión extra
        x = x.squeeze(1)
        x = self.end(x)
        return x


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)
        
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        attention = torch.nn.functional.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Modelos y optimizadores
embed_size = 128
heads = 4
gen = Generator(100, expression_data.shape[1], embed_size, heads)
disc = Discriminator(expression_data.shape[1] + methylation_data.shape[1])
gen_optimizer = optim.Adam(gen.parameters(), lr=0.0002)
disc_optimizer = optim.Adam(disc.parameters(), lr=0.0002)
criterion = nn.BCELoss()

# Entrenamiento
n_samples = combined_data.size(0)
batch_size = 64
n_epochs = 10000

for epoch in range(n_epochs):
    for idx in range(0, n_samples, batch_size):
        real_data = combined_data[idx:idx+batch_size]
        current_batch_size = real_data.size(0)
        real_labels = torch.ones(current_batch_size, 1)

        noise = torch.randn(current_batch_size, 100)
        fake_expression = gen(noise)
        fake_data = torch.cat((fake_expression, methylation_data[idx:idx+batch_size]), 1)
        fake_labels = torch.zeros(current_batch_size, 1)
        
        # Entrenar discriminador
        disc_optimizer.zero_grad()

        real_preds = disc(real_data)
        real_loss = criterion(real_preds, real_labels)
        
        fake_preds = disc(fake_data)
        fake_loss = criterion(fake_preds, fake_labels)

        disc_loss = real_loss + fake_loss
        disc_loss.backward()
        disc_optimizer.step()

        # Entrenar generador
        gen_optimizer.zero_grad()

        noise = torch.randn(current_batch_size, 100)
        fake_expression = gen(noise)
        fake_data = torch.cat((fake_expression, methylation_data[idx:idx+batch_size]), 1)
        fake_preds = disc(fake_data)

        gen_loss = criterion(fake_preds, real_labels)
        gen_loss.backward()
        gen_optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} | Disc Loss: {disc_loss.item()} | Gen Loss: {gen_loss.item()}")

# Generar datos de expresión sintética
noise = torch.randn(1, 100)
synthetic_expression = gen(noise)
print(synthetic_expression)



Epoch 100/10000 | Disc Loss: 5.945760494796559e-05 | Gen Loss: 10.515534400939941
Epoch 200/10000 | Disc Loss: 1.399466327711707e-05 | Gen Loss: 12.196635246276855
Epoch 300/10000 | Disc Loss: 5.123167284182273e-06 | Gen Loss: 13.017976760864258
Epoch 400/10000 | Disc Loss: 3.7316140151233412e-06 | Gen Loss: 14.001824378967285
Epoch 500/10000 | Disc Loss: 1.7040133570844773e-06 | Gen Loss: 14.61600399017334
Epoch 600/10000 | Disc Loss: 9.082845622288005e-07 | Gen Loss: 14.930032730102539
Epoch 700/10000 | Disc Loss: 3.8317355688377575e-07 | Gen Loss: 15.814483642578125
Epoch 800/10000 | Disc Loss: 1.9683631080624764e-07 | Gen Loss: 16.492481231689453
Epoch 900/10000 | Disc Loss: 3.140907551824057e-07 | Gen Loss: 16.668394088745117
Epoch 1000/10000 | Disc Loss: 1.4640127687925997e-07 | Gen Loss: 17.35086441040039
Epoch 1100/10000 | Disc Loss: 7.728576179033553e-08 | Gen Loss: 17.871023178100586
Epoch 1200/10000 | Disc Loss: 8.77089547657306e-08 | Gen Loss: 17.349214553833008
Epoch 1300/